In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv


In [2]:
df = pd.read_csv('/kaggle/input/emotions/text.csv')

In [3]:
df.head(10)

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
5,5,i was beginning to feel quite disheartened,0
6,6,i would think that whomever would be lucky eno...,2
7,7,i fear that they won t ever feel that deliciou...,1
8,8,im forever taking some time out to have a lie ...,5
9,9,i can still lose the weight without feeling de...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [5]:
len(df['label'].unique())

6

In [6]:
df['label'].unique()

array([4, 0, 2, 1, 5, 3])

In [7]:
df['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame
train_set, test_set = train_test_split(df, test_size=200000)

In [9]:
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_set['label'], num_classes=6)
test_labels = to_categorical(test_set['label'], num_classes=6)

2024-03-27 14:58:09.703348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 14:58:09.703493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 14:58:09.875203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
print(train_labels.shape)

(216809, 6)


In [11]:
class_totals = [121187,141067,34554,57317,47712,14972]  # Your class counts
total = sum(class_totals)
class_weights = {i: total / class_totals[i] for i in range(len(class_totals))}


In [12]:
df['label'].shape

(416809,)

In [13]:
train_labels.shape

(216809, 6)

In [14]:
train_labels

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Example preprocessing
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_set['text']) # Your 20 lines of text
tokenizer.fit_on_texts(test_set['text']) # Your 20 lines of text
sequences = tokenizer.texts_to_sequences(train_set['text'])
sequences2 = tokenizer.texts_to_sequences(test_set['text'])
data = pad_sequences(sequences, maxlen=100) # Example max length
Big = pad_sequences(sequences2, maxlen=100)



# Model architecture
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=64))
model.add(Dropout(0.3))  # Adjust dropout rate as needed
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))  # Adjust dropout rate as needed
model.add(Dense(6, activation='softmax'))



import tensorflow as tf

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.F1Score(average='weighted', threshold=None, name='f1_score', dtype=None)])



In [16]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping criteria
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
# Assuming 'test_data' and 'test_labels' are your test sets
history = model.fit(data, train_labels, epochs=2, validation_data=(Big, test_labels),class_weight=class_weights)


Epoch 1/2
6776/6776 ━━━━━━━━━━━━━━━━━━━━ 543s 79ms/step - f1_score: 0.7669 - loss: 3.6938 - val_f1_score: 0.9380 - val_loss: 0.1438
Epoch 2/2
6776/6776 ━━━━━━━━━━━━━━━━━━━━ 542s 80ms/step - f1_score: 0.9398 - loss: 0.7717 - val_f1_score: 0.9391 - val_loss: 0.1293


In [17]:
input_text = "I hate Sunday."
tokenizer = Tokenizer(num_words=13000)
tokenizer.fit_on_texts([input_text])

sequences1 = tokenizer.texts_to_sequences([input_text])

data = pad_sequences(sequences1, maxlen=100)
testprediction1 = model.predict(data)
# List of class names

class_names = ['sadness' , 'joy' , 'love', 'anger', 'fear', 'surprise']

# Assuming 'predicted_index' is the index of the predicted class obtained from the model
predicted_index = np.argmax(testprediction1, axis=1)  # For example, if predictions is the output of your model

# Get the predicted class name
predicted_class_name1 = class_names[predicted_index[0]]  # Use the first prediction as an example
print(f"Predicted class: {predicted_class_name1}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
Predicted class: anger


In [18]:
input_text = "I am not happy now."
tokenizer = Tokenizer(num_words=13000)
tokenizer.fit_on_texts([input_text])

sequences1 = tokenizer.texts_to_sequences([input_text])

data = pad_sequences(sequences1, maxlen=100)
testprediction1 = model.predict(data)
# List of class names

class_names = ['sadness' , 'joy' , 'love', 'anger', 'fear', 'surprise']

# Assuming 'predicted_index' is the index of the predicted class obtained from the model
predicted_index = np.argmax(testprediction1, axis=1)  # For example, if predictions is the output of your model

# Get the predicted class name
predicted_class_name1 = class_names[predicted_index[0]]  # Use the first prediction as an example
print(f"Predicted class: {predicted_class_name1}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: anger
